## Library Installation

In [ ]:
!pip install llama-agents llama-index-agent-openai llama-index-embeddings-openai

## Library Imports

In [ ]:
from llama_agents import (
    AgentService,
    ControlPlaneServer,
    SimpleMessageQueue,
    AgentOrchestrator,
)
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.tools import FunctionTool
from llama_index.llms.openai import OpenAI
import logging
from google.colab import userdata
import os
import nest_asyncio
nest_asyncio.apply()

## Enabling Logging

In [ ]:
logging.getLogger("llama_agents").setLevel(logging.INFO)

## API Setting

In [ ]:
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

## Defining Message Queue and Control Plane

In [ ]:
message_queue = SimpleMessageQueue()
control_plane = ControlPlaneServer(
    message_queue=message_queue,
    orchestrator=AgentOrchestrator(llm=OpenAI()),
)

## Tool Definition

In [ ]:
def get_the_syno() -> str:
    """Returns the word snyonym."""
    return "The synonym of the word Artificial Intelligence is: Expert Systems."

tool_1 = FunctionTool.from_defaults(fn=get_the_syno)

## Agent-as-a-Service Setup

In [ ]:
worker1 = FunctionCallingAgentWorker.from_tools([tool_1], llm=OpenAI())
agent1 = worker1.as_agent()

In [ ]:
agent_service_1 = AgentService(
    agent=agent1,
    message_queue=message_queue,
    description="Word Synonym Finder",
    service_name="synonym_finder",
    host = "localhost",
    port = 8003
)

In [ ]:
from llama_agents import ServerLauncher, CallableMessageConsumer

# Additional human consumer
def handle_result(message) -> None:
    print(f"Got result:", message.data)


# the final result is published to a "human" consumer
# so we define one to handle it!
human_consumer = CallableMessageConsumer(
    handler=handle_result, message_type="human"
)

# Define Launcher
launcher = ServerLauncher(
    [agent_service_1],
    control_plane,
    message_queue,
    additional_consumers=[human_consumer]
)

launcher.launch_servers()